In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel, Matern


from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')

In [4]:
X_data = pd.read_csv("starting_kit/data/protein_train.data", sep=" ", header=None)
y_data = pd.read_csv("starting_kit/data/protein_train.solution", sep=" ", header=None)

X_test_data = pd.read_csv("starting_kit/data/protein_test.data", sep=" ", header=None)
X_valid_data = pd.read_csv("starting_kit/data/protein_valid.data", sep=" ", header=None)

In [5]:
X_data

,0,1,2,3,4,5,6,7,8,9,...,942,943,944,945,946,947,948,949,950,951
0,0.068,0.068,0.085,0.068,0.000,0.051,0.017,0.000,0.051,0.051,...,8.996,-0.156,-2.296,-5.124,4.268,-0.248,-4.644,2.697,1.124,2.635
1,0.094,0.069,0.064,0.099,0.039,0.073,0.043,0.013,0.034,0.077,...,27.622,2.517,3.343,-19.543,17.150,-11.894,-14.239,-0.212,6.017,8.955
2,0.067,0.062,0.054,0.132,0.045,0.060,0.038,0.018,0.018,0.054,...,64.266,11.886,-0.480,-44.047,39.708,-31.910,-37.893,-5.492,11.690,20.689
3,0.063,0.056,0.071,0.162,0.036,0.058,0.062,0.022,0.036,0.036,...,80.416,21.129,4.508,-55.792,32.963,-26.239,-62.055,-3.766,10.171,20.835
4,0.081,0.085,0.078,0.104,0.030,0.078,0.033,0.000,0.019,0.074,...,34.747,2.485,-2.486,-25.222,22.287,-13.880,-23.592,5.089,10.324,9.218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53113,0.088,0.095,0.067,0.091,0.047,0.091,0.030,0.000,0.032,0.060,...,62.061,11.254,-11.063,-41.835,41.653,-17.432,-38.004,5.285,17.813,21.263
53114,0.054,0.034,0.041,0.095,0.054,0.027,0.034,0.041,0.020,0.074,...,16.781,4.918,0.903,-3.647,8.940,-8.705,-6.220,-5.351,-0.587,6.750
53115,0.109,0.088,0.076,0.086,0.035,0.086,0.033,0.002,0.029,0.064,...,67.318,13.292,-8.059,-52.886,39.761,-20.426,-41.685,2.594,15.800,22.920
53116,0.074,0.047,0.098,0.195,0.070,0.051,0.060,0.023,0.019,0.028,...,24.277,5.070,2.114,-13.588,12.932,-11.197,-19.363,-2.118,1.972,5.346


In [6]:
X_data2 = X_data.iloc[:15].copy()
y_data2 = y_data.iloc[:15].copy()
X_train, X_test, y_train, y_test = train_test_split(X_data2, y_data2,random_state=0, test_size=0.2)
scaler = RobustScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
pca = PCA(n_components = 0.95)
pca.fit(X_train_scaled)
X_train_scaled = pca.transform(X_train_scaled)
X_test_scaled = pca.transform(X_test_scaled)


In [104]:
# XGboost
tuned_parameters = [{'max_depth': [5,10, 15, 20, 25, 30],'learning_rate':[0.001, 0.01, 0.1, 0.5], 'n_estimators': [100,150,200, 250, 300]}]
clas = GridSearchCV(xgb.XGBClassifier(), tuned_parameters, cv=4, n_jobs=-1)
clas.fit(X_train_scaled, y_train)
y_true, y_pred2 = y_test, clas.predict(X_test_scaled)
    
print('The best hyper-parameters for XGBBoost are: ',clas.best_params_)

In [62]:
# Random Forests
tuned_parameters = [{'max_depth': [5,10, 15, 20, 50, 70], 'n_estimators': [10, 25, 50, 100,150, 200, 250]}]
clas_rf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=4, n_jobs=-1)
clas_rf.fit(X_train_scaled, y_train)
y_true, y_pred5 = y_test, clas_rf.predict(X_test_scaled)
    
print('The best hyper-parameters for Random Forests are: ',clas_rf.best_params_)

The best hyper-parameters for Random Forests are:  {'max_depth': 5, 'n_estimators': 10}


In [8]:
#KNN
tuned_parameters = [{'n_neighbors': [1,2,3,4,5,10,15,20], 'p': [1,2]}]
model = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=4)
model.fit(X_train_scaled, y_train)
    
print('The best hyper-parameters for KNN are: ', model.best_params_)

The best hyper-parameters for KNN are:  {'n_neighbors': 1, 'p': 1}


In [64]:
# SVM
tuned_parameters = [{'kernel': ['linear', 'rbf', 'poly'], 'C':[1, 2, 3, 5, 6, 7, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1]}]
svm_clas = GridSearchCV(SVC(), tuned_parameters, cv=4, n_jobs=-1)
svm_clas.fit(X_train_scaled, y_train)
    
print('The best hyper-parameters for SVR are: ', svm_clas.best_params_)

The best hyper-parameters for SVR are:  {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}


In [101]:
# Decision Tree
tuned_parameters = [{'max_depth': [1,2,3,4,5,10, 15, 20, 25, 50, 100,200]}]
clas_dt = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=4, n_jobs=-1)
clas_dt.fit(X_train_scaled, y_train)
    
print('The optimum max_depth for Decision Tree is: ', clas_dt.best_params_ )

ValueError: Found input variables with inconsistent numbers of samples: [31870, 42494]

In [69]:
# GP
kernel1 = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
kernel2 = C() + Matern(length_scale=0.5, nu=3/2) 
kernel3 = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) +  C(1.0, (1e-3, 1e3))* Matern(length_scale=2, nu=3/2)
kernel4 = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e+1))

tuned_parameters = [{'kernel': [kernel1,kernel2,kernel3,kernel4], 
                   'n_restarts_optimizer':[5]}]
gpclas = GridSearchCV(GaussianProcessClassifier(), tuned_parameters, cv=4, n_jobs=-1)
gpclas.fit(X_train_scaled, y_train)
    
print('The best hyper-parameters for GPR are: ', gpclas.best_params_ )

The best hyper-parameters for GPR are:  {'kernel': 1**2 * RBF(length_scale=10), 'n_restarts_optimizer': 5}


In [70]:
# XGBoost
xgclas = xgb.XGBClassifier(learning_rate=0.001, max_depth=5, n_estimators=100, random_state = 0)
xgclas.fit(X_train_scaled, y_train, n_jobs=-1)
y_pred1 = xgclas.predict(X_test_scaled)              
print('XGBoost Classifier :accuracy_score', accuracy_score(y_test,y_pred1))

# Decision Tree
clas_dt = DecisionTreeClassifier(random_state=0, max_depth = 4)
clas_dt.fit(X_train_scaled,y_train, n_jobs=-1)
y_pred2 = clas_dt.predict(X_test_scaled)        # Decision Tree
print('Decision Tree Classifier :accuracy_score', accuracy_score(y_test,y_pred2))

# Random Forests
regr_rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=10)
regr_rf.fit(X_train_scaled, y_train)
y_pred3 = regr_rf.predict(X_test_scaled)           
print('Random Forests Classifier -accuracy_score:', accuracy_score(y_test,y_pred3))

# Logistic Regression
logclas = LogisticRegression()
logclas.fit(X_train_scaled, y_train, n_jobs=-1)
y_pred4 = logclas.predict(X_test_scaled)           
print('Logistic Regressor :accuracy_score', accuracy_score(y_test,y_pred4))

# KNN
neigh = KNeighborsClassifier(n_neighbors = 2, metric = 'minkowski', p = 1)
neigh.fit(X_train_scaled, y_train, n_jobs=-1)
y_pred5 = neigh.predict(X_test_scaled) 
print('KNN Classifier :accuracy_score:', accuracy_score(y_test,y_pred5))

# SVM
svm_clas = SVC(gamma=1, kernel = 'linear', C=0.0001)
svm_clas.fit(X_train_scaled, y_train, n_jobs=-1)
y_pred6 = svm_clas.predict(X_test_scaled)     # SVR
print('SVR Classifier :accuracy_score:',accuracy_score(y_test,y_pred6))

# GPR
gp = GaussianProcessClassifier(kernel=kernel1, n_restarts_optimizer=5)
gp.fit(X_train_scaled, y_train, n_jobs=-1)
y_pred8 = gp.predict(X_test_scaled)         
print('GP Classifier :accuracy_score:', accuracy_score(y_test,y_pred8))

[19:21:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Classifier :accuracy_score 1.0
Decision Tree Classifier :accuracy_score 1.0
Random Forests Classifier -accuracy_score: 1.0
Logistic Regressor :accuracy_score 1.0
KNN Classifier :accuracy_score: 1.0
SVR Classifier :accuracy_score: 1.0
GP Classifier :accuracy_score: 1.0
